In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

## Difference-in-difference (DID)
- DID (real panel): AER::Fatalities
-     Outcome: traffic fatality rate per 10,000 pop
-     Policy (simple): “drinkage >= 21” indicator (raised drinking age)

In [ ]:
fatal = sm.datasets.get_rdataset("Fatalities", package="AER").data

# Make outcome = fatalities per 10,000 population
fatal["frate"] = fatal["fatal"] / fatal["pop"] * 10000

# Simple policy indicator: treated when legal drinking age reaches 21
fatal["policy"] = (fatal["drinkage"] >= 21).astype(int)

In [ ]:
# Keep only needed columns
fatal = fatal[["state", "year", "frate", "policy"]].dropna()
fatal["state"] = fatal["state"].astype(str)
fatal["year"] = fatal["year"].astype(int)

fatal

In [ ]:
# (A) Minimal DID plot: average frate by year and policy status
avg = fatal.groupby(["year", "policy"])["frate"].mean().reset_index()
fig, ax = plt.subplots()

for d, g in avg.groupby("policy"):
    ax.plot(g["year"], g["frate"], marker="o", label=f"policy={d}")

ax.set_title("DID sketch: Fatality rate by policy status over time")
ax.set_xlabel("Year")
ax.set_ylabel("Fatalities per 10,000")
ax.legend()

plt.vlines(1985,1.9,3.2, color="r")
plt.show()

In [ ]:
# (B) Minimal DID regression: TWFE (state & year fixed effects) with cluster SEs by state
# Formula form with dummies for state/year:
did_fit = ols("frate ~ policy + C(state) + C(year)", data=fatal).fit(
    cov_type="cluster", cov_kwds={"groups": fatal["state"]}
)
print("\n=== DID: TWFE with cluster-robust SEs by state ===")
print(did_fit.summary().tables[1])
print("\nInterpretation: 'policy' is the DID estimate under parallel trends.\n")

# Regression Discontinuity Analysis

### First make some fake data

In [ ]:
# --- Simulate data ---
n = 1000
X = np.random.uniform(0, 100, n)          # running variable
cutoff = 50                               # treatment threshold
D = (X >= cutoff).astype(int)             # treatment indicator

# Smooth outcome trend + treatment jump + random noise
Y = 0.5 * X + 10*D + np.random.normal(0, 5, n)

data = pd.DataFrame({"Y": Y, "X": X, "D": D})

In [ ]:
plt.scatter(X, Y, alpha=0.3, label="Data")
plt.axvline(cutoff, color="red", linestyle="--", label="Cutoff (c=50)")
plt.xlabel("Running variable (X)")
plt.ylabel("Outcome (Y)")
plt.title("Regression Discontinuity: Simulated Data")
plt.legend()

In [ ]:
# --- 4. Local linear regression around cutoff ---
bandwidth = 10

# Keep only observations close to the cutoff (±10 points)
subset = data[(data["X"] > cutoff - bandwidth) & (data["X"] < cutoff + bandwidth)].copy()

# Center X around the cutoff
subset["X_centered"] = subset["X"] - cutoff

# Fit local linear model with different slopes on each side
model = smf.ols("Y ~ D + X_centered + D:X_centered", data=subset).fit()

print(model.summary())

x_grid = np.linspace(cutoff - bandwidth, cutoff + bandwidth, 200)
df_pred = pd.DataFrame({"X": x_grid})
df_pred["D"] = (df_pred["X"] >= cutoff).astype(int)
df_pred["X_centered"] = df_pred["X"] - cutoff
df_pred["Y_pred"] = model.predict(df_pred)

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(subset["X"], subset["Y"], alpha=0.4, label="Data (within bandwidth)")
plt.plot(df_pred["X"], df_pred["Y_pred"], color="black", lw=2, label="Fitted lines")
plt.axvline(cutoff, color="red", linestyle="--", lw=2, label=f"Cutoff = {cutoff}")
plt.xlabel("Running variable (X)")
plt.ylabel("Outcome (Y)")
plt.title("Regression Discontinuity (Local Linear Fit)")
plt.legend()
plt.show()

In [ ]:
tau = model.params["D"]
print(f"\nEstimated treatment effect (jump at cutoff): {tau:.2f}")